### Start with importing class

In [1]:
from desummation import Desummation
import numpy as np
dsm = Desummation(frobenius=False)

C:\Users\Саша Кукунцев\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create new matrix
$$
    A = \begin{pmatrix}
    3 & 7 & 2 & 0 \\
    -4 & 2 & 0 & -3 \\
    5 & 0 & 2 & -1 \\
    5 & -5 & -2 & -4 \\
    \end{pmatrix} 
$$

In [2]:
A = [[3, 7, 2, 0], [-4, 2, 0, -3], [5, 0, 2, -1], [5, -5, -2, -4]] 

### Now fit some random matrices to this matrix

In [3]:
dsm.fit(A, 8)

### Get all the information you might need

In [4]:
dsm.weights()

[-1.2050535534792859,
 -1.7716763717077209,
 1.452825456808684,
 1.4224859182671477,
 -1.606490406459356,
 -0.5262668143480465,
 -1.0917427037058465,
 -0.1794805667112671]

### Now let's see which matrix we got

In [5]:
dsm.predict(A)

array([[ 4.10126585,  0.42840672,  0.92154097, -0.58236963],
       [ 2.16004584, -2.45695537,  6.61228674,  1.81777951],
       [ 0.8458048 , -1.90768291, -2.29805276, -1.49476659],
       [-0.8412738 , -3.61779018, -1.95726077, -1.36709211]])

In [6]:
np.array(A)

array([[ 3,  7,  2,  0],
       [-4,  2,  0, -3],
       [ 5,  0,  2, -1],
       [ 5, -5, -2, -4]])

In [7]:
dsm.error()

18.713472499318648

### This is not very good approximation.
#### Certain ways to improve this:
- ##### make more trials for weights searching (using weights_old.py)
- ##### `make more random matrices`
- ##### change metric between matrices
- ##### change distribution of elements in matrices

In [8]:
dsm.fit(A, 16, n_trials=2000, distribution='exponential', scale=1)

In [9]:
dsm.predict(A)

array([[ 3.05526903,  6.28878661,  3.3622227 , -0.0655204 ],
       [-2.72399592,  3.39279763, -0.39034037, -2.57913701],
       [ 3.58614135,  0.52833864,  2.07295215, -1.17704185],
       [ 4.09971326, -4.85845134, -3.2989131 , -4.87177517]])

In [10]:
np.array(A)

array([[ 3,  7,  2,  0],
       [-4,  2,  0, -3],
       [ 5,  0,  2, -1],
       [ 5, -5, -2, -4]])

In [11]:
dsm.error()

3.4521690080402267

In [12]:
dsm.weights()

[-1.181808591628415,
 -3.4599760297534092,
 0.2878106845615882,
 -1.8568038314308914,
 0.6290504454683159,
 2.1792587436684734,
 -3.494782208509391,
 -1.7486985934939208,
 3.2147060908656186,
 -1.88177511853001,
 1.540429819399181,
 -0.6341104707362737,
 1.3703451831917288,
 0.9255305947322183,
 3.221359735801432,
 -1.0286403621045794]

# What are the inferences?

### You can experiment with it yourself, but judging by my intuition and my experiments:
- #### Almost always we will find weights with (frobenius norm) loss close to 0 `if` we use $n^2$ weights (tested only for square matrix for now). I think that it may be some sort of `solution to a linear system`. You may understand me beforehand if you remember the basis for matrices(the one consisting of $E_{ij}$ matrices)
- #### I think a lot about applying this to ML and DL and came up to some insight: [you can see this page](https://weightagnostic.github.io/) or trust my words. The experiment showcased on this page focused on utilizing random weights in ANNs `without` any explicit `weight training`, but `rather allowing the weights between neuron connections to be learned`. This approach offers a more cost-effective training method while still achieving impressive capabilities. This just reminds me of my problem.

## Now let's get some information about loss function for some fixed basis
### Is it convex? For someone who already knows the answer, you can go just down below for an explanation.

### First, start with simple one:
$$
A = \begin{pmatrix}
2 & -1 \\
-5 & 4 \\
\end{pmatrix}
$$


In [13]:
import plotly.graph_objects as go
dsm_convexity2 = Desummation()
A1 = [[2, -1], [-5, 4]]
dsm_convexity2.fit(A1, 2)

### Now define a function for plotting
- #### We will need a meshgrid of x and y.
    - ##### I will be using only 2 random matrices for good visualization (x and y coordinates).
    - ##### Weights are programmed to be found from $2 \cdot min$ to $2 \cdot max$ value of matrix A, but maybe this is wrong. I don't know it yet.
- #### Also a function that will return an error, for that I will calculate the frobenius norm loss.

In [14]:
n = 200
w1 = np.linspace(-20, 20, n)
w2 = np.linspace(-20, 20, n)

In [15]:
def frobenius_loss(x, y, dsm_object, target, distance='fro'):
    B = np.tensordot(np.stack([x, y]), dsm_object.matrices(), axes=1)
    return np.linalg.norm(target - B, ord=distance)

In [16]:
def plot(w1, w2, Z):
    fig = go.Figure(data=[go.Surface(x=w1, y=w2, z=Z)], layout=go.Layout(width=600, height=400))

    # Set layout options
    fig.update_layout(
        title='3D Plot of Loss Function',
        scene=dict(
            xaxis_title='w1',
            yaxis_title='w2',
            zaxis_title='Loss',
            aspectratio=dict(x=1, y=1, z=1),
            camera=dict(
                eye=dict(x=1, y=1, z=1)
            )
        ),
        autosize=True
    )
    argmin = np.argmin(Z)
    row_index = argmin // n
    col_index = argmin % n

    print(f'Minimum value obtained at {np.min(Z)} with weights: w1:{w1[col_index]} and w2:{w2[row_index]}')
    fig.show()


In [17]:
Z = np.array([[frobenius_loss(x=x, y=y, dsm_object=dsm_convexity2, target=A1) for x in w1] for y in w2])
#plot(w1, w2, Z)

![plot1](plot1.png)

### I think it's a strike!!!
#### It's `clearly convex` function! And only with 2 weights. Case was very simple, but you can modify values in A, call dsm_convexety.fit() again to update random matrices, see if anything changes.

### Now let's try more difficult one:
#### This time convexity is quite questionable
$$
A = \begin{pmatrix}
2 & -5 & 4 \\
-3 & -3 & 2 \\
1 & 2 & 6 \\
\end{pmatrix}
$$


In [18]:
dsm_convexity3 = Desummation()
A2 = [[2, -5, 4], [-3, -3, 2], [1, 2, 6]]
dsm_convexity3.fit(A2, 2, distribution = 'normal')

In [19]:
n = 100
w1 = np.linspace(-12, 12, n)
w2 = np.linspace(-12, 12, n)

In [20]:
Z = np.array([[frobenius_loss(x=x, y=y, dsm_object=dsm_convexity3, target=A2) for x in w1] for y in w2])
#plot(w1, w2, Z)

Minimum value obtained at 10.300788969401598 with weights: w1:-0.3636363636363633 and w2:-0.3636363636363633


[array([[-0.29759239, -1.40926714, -1.34122279],
       [-0.26013598,  0.43407102,  1.80260928],
       [-1.61428976,  0.31691087, -1.16048182]]), array([[ 0.31995238, -0.46583704,  0.32864676],
       [ 1.45977866, -0.70001791, -1.06900191],
       [ 0.32773005, -0.92941536,  0.1821938 ]])]


![plot2](plot2.png)

### This is convex too! You can try various shapes of A, various distributions, but there wouldn't be a case of not convex plot.
#### This is all because of [convexity of frobenius norm](https://ics.uci.edu/~xhx/courses/ConvexOpt/convex_functions.pdf)!
So we can just go with how many weights we want and will always find a best approximation for our matrix.
## Now it's time to reveal the truth...

## There is actually a *numerical solution*. And, as we already know, `distinct` which is due to convexity.

#### It took 20 seconds for matrix with shape (4, 4) and with 16 random matrices to find a solution, which can still appear to be not the best one.
#### Now let me solve it in <0.1 second with brilliant accuracy:

In [153]:
A = [[3, 7, 2, 0], [-4, 2, 0, -3], [5, 0, 2, -1], [5, -5, -2, -4]] 
dsm_new = Desummation(frobenius=True)
print(np.array(A), dsm_new.fit_predict(A, 16), dsm_new.weights(), dsm_new.error(), sep='\n')

[[ 3  7  2  0]
 [-4  2  0 -3]
 [ 5  0  2 -1]
 [ 5 -5 -2 -4]]
[[ 3.00000000e+00  7.00000000e+00  2.00000000e+00  0.00000000e+00]
 [-4.00000000e+00  2.00000000e+00  2.66453526e-15 -3.00000000e+00]
 [ 5.00000000e+00  6.21724894e-15  2.00000000e+00 -1.00000000e+00]
 [ 5.00000000e+00 -5.00000000e+00 -2.00000000e+00 -4.00000000e+00]]
[-7.74717883  1.53876264 -1.05830028  3.17029003 10.07115783  0.79503887
 -2.39654625  9.58414218 -2.91367073  8.08327061  1.00258966  2.03499058
  7.78237368 -1.48187841 -3.6678655  -6.97975278]
4.805406418616647e-14


### For now it works with frobenius norm between matrices and maybe there is closer in some metric.
### But when amount is equals to a number of elements in matrix it finds the best approximation possible

# Why this works:


### I'll explain it in the case of 3x3 matrix, but all stays the same for any shape

### (*Definition*)
$$
A = \begin{pmatrix}
p_1 & p_2 & p_3 \\
p_4 & p_5 & p_6 \\
p_7 & p_8 & p_9 \\
\end{pmatrix}_{3 \times 3} = w_1 \cdot \begin{pmatrix}
a^1_{11} & a^1_{12} & a^1_{13} \\
a^1_{21} & a^1_{22} & a^1_{23} \\
a^1_{31} & a^1_{32} & a^1_{33} \\
\end{pmatrix}_{3 \times 3}+ \ldots + w_k \cdot \begin{pmatrix}
a^k_{11} & a^k_{12} & a^k_{13} \\
a^k_{21} & a^k_{22} & a^k_{23} \\
a^k_{31} & a^k_{32} & a^k_{33} \\
\end{pmatrix}_{3 \times 3}
$$
### This is equivalent to this *system of linear equations*:
$$
w_1 \cdot a^1_{11} + w_2 \cdot a^2_{11} + \ldots + w_k \cdot a^k_{11} = p_1 \
$$
$$
\vdots
$$
$$
w_1 \cdot a^1_{33} + w_2 \cdot a^2_{33} + \ldots + w_k \cdot a^k_{33} = p_9
$$
- ##### *9* equations (n $\cdot$ m in general case)
- ##### *k* weights
- ##### System is *overdefined*
    - ##### There is already a solution: [you can check this Wikipedia page with explicit answer for overdefined systems](https://en.wikipedia.org/wiki/Overdetermined_system#Approximate_solutions)
    - ##### This solution is `least squares method`
    - ##### And for k = n $\cdot$ m there is the best possible solution

# Now this code becomes really fast and I will certainly try to implement it later in the future. `Stay tuned for updates!`